# API Interaction Notebook

This notebook automates the setup and interaction with the FastAPI service and MongoDB database. It performs the following tasks:

1. **Start MongoDB** using Docker.
2. **Start FastAPI** using `uvicorn` in a subprocess.
3. **Perform CRUD operations** on patient records.
4. **Shut down** both services gracefully.

In [1]:
# Import necessary libraries
import subprocess
import time
import requests
import json
import docker
from docker.errors import APIError, NotFound
import os

# Suppress insecure request warnings (optional)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## 1. Start MongoDB Using Docker

In [2]:
# Function to start MongoDB container
def start_mongodb_container(container_name="mongodb", port=27017, db_name="patient_db"):
    client = docker.from_env()
    try:
        # Check if container already exists
        container = client.containers.get(container_name)
        print(f"Container '{container_name}' already exists. Starting it...")
        container.start()
    except NotFound:
        print(f"Container '{container_name}' not found. Creating and starting it...")
        try:
            container = client.containers.run(
                "mongo:latest",
                name=container_name,
                ports={"27017/tcp": port},
                environment={"MONGO_INITDB_DATABASE": db_name},
                detach=True
            )
            print(f"MongoDB container '{container_name}' started.")
        except APIError as e:
            print(f"Failed to start MongoDB container: {e.explanation}")
            raise e
    return container

# Start MongoDB
mongodb_container = start_mongodb_container()

# Wait for MongoDB to initialize
print("Waiting for MongoDB to initialize...")
time.sleep(10)  # Adjust if necessary
print("MongoDB is ready.")

## 2. Start FastAPI Service Using `uvicorn` in a Subprocess

In [3]:
# Function to start FastAPI using uvicorn
def start_fastapi(host="0.0.0.0", port=8000):
    # Ensure the virtual environment is activated and FastAPI app is accessible
    cmd = ["uvicorn", "app.main:app", "--host", host, "--port", str(port), "--reload"]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("FastAPI service started.")
    return process

# Start FastAPI
fastapi_process = start_fastapi()

# Wait for FastAPI to start
print("Waiting for FastAPI to start...")
time.sleep(5)  # Adjust if necessary
print("FastAPI is ready.")

## 3. Perform CRUD Operations via FastAPI

In [4]:
# Define the base URL of the API
BASE_URL = "http://localhost:8000"

### 3.1. Create a New Patient Record

In [5]:
# Define a new patient record
new_record = {
    "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
    "info": {
        "PatientID": "71054xfdsar",
        "PatientName": "SMITH^JANE",
        "PatientAge": "012Y",
        "StudyDate": "20240923"
    },
    "femur": {
        "Right femur": "41.8 cm",
        "Left femur": "42.0 cm",
        "Difference": "00.2 cm, left longer 0.5%"
    },
    "tibia": {
        "Right tibia": "34.5 cm",
        "Left tibia": "34.3 cm",
        "Difference": "00.2 cm, right longer 0.6%"
    },
    "total": {
        "Total right": "76.3 cm",
        "Total left": "76.3 cm",
        "Difference": "00.0 cm, equal 0.0%"
    },
    "pixel_distance": {
        "Left femur": 1892,
        "Left tibia": 1544,
        "Right femur": 1886,
        "Right tibia": 1555
    },
    "details": {
        "AccessionNumber": "100876169",
        "StudyDescription": "XR HIPS TO ANKLES LEG MEASUREMENTS",
        "SeriesDescription": "Lower limbs",
        "BodyPartExamined": "LEG",
        "FieldOfViewDimensions": "[975, 391]",
        "StationName": "EOSRM7"
    }
}

# Send POST request to create a new patient record
response = requests.post(f"{BASE_URL}/patients/", json=new_record)

if response.status_code == 201:
    print("Patient record created successfully:")
    print(json.dumps(response.json(), indent=4))
else:
    print("Error creating patient record:", response.text)

Patient record created successfully:
{
    "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
    "info": {
        "PatientID": "71054xfdsar",
        "PatientName": "SMITH^JANE",
        "PatientAge": "012Y",
        "StudyDate": "20240923"
    },
    "femur": {
        "Right femur": "41.8 cm",
        "Left femur": "42.0 cm",
        "Difference": "00.2 cm, left longer 0.5%"
    },
    "tibia": {
        "Right tibia": "34.5 cm",
        "Left tibia": "34.3 cm",
        "Difference": "00.2 cm, right longer 0.6%"
    },
    "total": {
        "Total right": "76.3 cm",
        "Total left": "76.3 cm",
        "Difference": "00.0 cm, equal 0.0%"
    },
    "pixel_distance": {
        "Left femur": 1892,
        "Left tibia": 1544,
        "Right femur": 1886,
        "Right tibia": 1555
    },
    "details": {
        "AccessionNumber": "100876169",
        "StudyDescription": "XR HIPS TO ANKLES LEG MEASUREMENTS",
        "SeriesDescription": "Lower limbs",
        "Bo

### 3.2. Get Patient Record by Study ID

In [6]:
# Define the study_id to retrieve
study_id = "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139"

# Send GET request to retrieve the patient record
response = requests.get(f"{BASE_URL}/patients/{study_id}")

if response.status_code == 200:
    print(f"Details of patient record {study_id}:")
    print(json.dumps(response.json(), indent=4))
else:
    print("Error fetching patient record:", response.text)

Details of patient record 61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139:
{
    "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
    "info": {
        "PatientID": "71054xfdsar",
        "PatientName": "SMITH^JANE",
        "PatientAge": "012Y",
        "StudyDate": "20240923"
    },
    "femur": {
        "Right femur": "41.8 cm",
        "Left femur": "42.0 cm",
        "Difference": "00.2 cm, left longer 0.5%"
    },
    "tibia": {
        "Right tibia": "34.5 cm",
        "Left tibia": "34.3 cm",
        "Difference": "00.2 cm, right longer 0.6%"
    },
    "total": {
        "Total right": "76.3 cm",
        "Total left": "76.3 cm",
        "Difference": "00.0 cm, equal 0.0%"
    },
    "pixel_distance": {
        "Left femur": 1892,
        "Left tibia": 1544,
        "Right femur": 1886,
        "Right tibia": 1555
    },
    "details": {
        "AccessionNumber": "100876169",
        "StudyDescription": "XR HIPS TO ANKLES LEG MEASUREMENTS",
        "Seri

### 3.3. Update Patient Record

In [7]:
# Define the updated patient record
updated_record = {
    "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
    "info": {
        "PatientID": "71054xfdsar",
        "PatientName": "SMITH^JANE",
        "PatientAge": "013Y",  # Updated age
        "StudyDate": "20240923"
    },
    "femur": {
        "Right femur": "41.9 cm",  # Updated measurement
        "Left femur": "42.1 cm",
        "Difference": "00.2 cm, left longer 0.5%"
    },
    "tibia": {
        "Right tibia": "34.6 cm",
        "Left tibia": "34.4 cm",
        "Difference": "00.2 cm, right longer 0.6%"
    },
    "total": {
        "Total right": "76.5 cm",
        "Total left": "76.5 cm",
        "Difference": "00.0 cm, equal 0.0%"
    },
    "pixel_distance": {
        "Left femur": 1900,
        "Left tibia": 1550,
        "Right femur": 1890,
        "Right tibia": 1560
    },
    "details": {
        "AccessionNumber": "100876169",
        "StudyDescription": "XR HIPS TO ANKLES LEG MEASUREMENTS",
        "SeriesDescription": "Lower limbs",
        "BodyPartExamined": "LEG",
        "FieldOfViewDimensions": "[975, 391]",
        "StationName": "EOSRM7"
    }
}

# Send PUT request to update the patient record
response = requests.put(f"{BASE_URL}/patients/{study_id}", json=updated_record)

if response.status_code == 200:
    print("Patient record updated successfully:")
    print(json.dumps(response.json(), indent=4))
else:
    print("Error updating patient record:", response.text)

Patient record updated successfully:
{
    "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
    "info": {
        "PatientID": "71054xfdsar",
        "PatientName": "SMITH^JANE",
        "PatientAge": "013Y",
        "StudyDate": "20240923"
    },
    "femur": {
        "Right femur": "41.9 cm",
        "Left femur": "42.1 cm",
        "Difference": "00.2 cm, left longer 0.5%"
    },
    "tibia": {
        "Right tibia": "34.6 cm",
        "Left tibia": "34.4 cm",
        "Difference": "00.2 cm, right longer 0.6%"
    },
    "total": {
        "Total right": "76.5 cm",
        "Total left": "76.5 cm",
        "Difference": "00.0 cm, equal 0.0%"
    },
    "pixel_distance": {
        "Left femur": 1900,
        "Left tibia": 1550,
        "Right femur": 1890,
        "Right tibia": 1560
    },
    "details": {
        "AccessionNumber": "100876169",
        "StudyDescription": "XR HIPS TO ANKLES LEG MEASUREMENTS",
        "SeriesDescription": "Lower limbs",
        "Bo

### 3.4. List All Patient Records

In [8]:
# Send GET request to list all patient records
response = requests.get(f"{BASE_URL}/patients/")

if response.status_code == 200:
    print("List of all patient records:")
    print(json.dumps(response.json(), indent=4))
else:
    print("Error listing patient records:", response.text)

List of all patient records:
[
    {
        "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
        "info": {
            "PatientID": "71054xfdsar",
            "PatientName": "SMITH^JANE",
            "PatientAge": "013Y",
            "StudyDate": "20240923"
        },
        "femur": {
            "Right femur": "41.9 cm",
            "Left femur": "42.1 cm",
            "Difference": "00.2 cm, left longer 0.5%"
        },
        "tibia": {
            "Right tibia": "34.6 cm",
            "Left tibia": "34.4 cm",
            "Difference": "00.2 cm, right longer 0.6%"
        },
        "total": {
            "Total right": "76.5 cm",
            "Total left": "76.5 cm",
            "Difference": "00.0 cm, equal 0.0%"
        },
        "pixel_distance": {
            "Left femur": 1900,
            "Left tibia": 1550,
            "Right femur": 1890,
            "Right tibia": 1560
        },
        "details": {
            "AccessionNumber": "100876169",
   

### 3.5. List Patient Records Filtered by PatientID

In [9]:
# Define the PatientID to filter
patient_id = "71054xfdsar"

# Send GET request to list patient records filtered by PatientID
response = requests.get(f"{BASE_URL}/patients/", params={"patient_id": patient_id})

if response.status_code == 200:
    print(f"List of patient records for PatientID {patient_id}:")
    print(json.dumps(response.json(), indent=4))
else:
    print("Error filtering patient records:", response.text)

List of patient records for PatientID 71054xfdsar:
[
    {
        "study_id": "61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139",
        "info": {
            "PatientID": "71054xfdsar",
            "PatientName": "SMITH^JANE",
            "PatientAge": "013Y",
            "StudyDate": "20240923"
        },
        "femur": {
            "Right femur": "41.9 cm",
            "Left femur": "42.1 cm",
            "Difference": "00.2 cm, left longer 0.5%"
        },
        "tibia": {
            "Right tibia": "34.6 cm",
            "Left tibia": "34.4 cm",
            "Difference": "00.2 cm, right longer 0.6%"
        },
        "total": {
            "Total right": "76.5 cm",
            "Total left": "76.5 cm",
            "Difference": "00.0 cm, equal 0.0%"
        },
        "pixel_distance": {
            "Left femur": 1900,
            "Left tibia": 1550,
            "Right femur": 1890,
            "Right tibia": 1560
        },
        "details": {
            "AccessionNum

### 3.6. Delete a Patient Record

In [10]:
# Send DELETE request to delete the patient record
response = requests.delete(f"{BASE_URL}/patients/{study_id}")

if response.status_code == 204:
    print(f"Patient record {study_id} deleted successfully.")
else:
    print("Error deleting patient record:", response.text)

Patient record 61928-1.2.250.1.118.3.1305.235.1.8008.46.1727122139 deleted successfully.


## 4. Shut Down Services

In [11]:
# Function to stop FastAPI subprocess
def stop_fastapi(process):
    if process.poll() is None:
        print("Shutting down FastAPI service...")
        process.terminate()
        try:
            process.wait(timeout=5)
            print("FastAPI service terminated.")
        except subprocess.TimeoutExpired:
            process.kill()
            print("FastAPI service killed.")
    else:
        print("FastAPI service is not running.")

# Function to stop MongoDB container
def stop_mongodb_container(container):
    try:
        container.stop()
        print(f"MongoDB container '{container.name}' stopped.")
    except APIError as e:
        print(f"Failed to stop MongoDB container: {e.explanation}")

# Stop FastAPI
stop_fastapi(fastapi_process)

# Stop MongoDB
print(f"Stopping MongoDB container '{mongodb_container.name}'...")
stop_mongodb_container(mongodb_container)

Shutting down FastAPI service...
FastAPI service terminated.
Stopping MongoDB container 'mongodb'...
MongoDB container 'mongodb' stopped.
